In [1]:
#This is my copy of the JSON mini-project exercise
#Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,

# 1. Find the 10 countries with most projects
# 2. Find the top 10 major project themes (using column 'mjtheme_namecode')
# 3. In 2. above you will notice that some entries have only the code and the name is missing. 
# Create a dataframe with the missing names filled in.

#import modules
import pandas as pd
import json
from pandas.io.json import json_normalize
from IPython.display import display #this module enables us to see all columns in a pandas dataframe 

#load JSON file into Python as Pandas dataframe
wb = pd.read_json('data/world_bank_projects.json')




In [2]:
#enable all dataframe columns to be seen with the following two lines of code
#pd.options.display.max_columns = None
#display(wb)
###########
#get top 10 countries in terms of # of projects
#count() means counting non-missing entries
wb.groupby('countryname')['project_name'].count().sort_values(ascending = False).head(10)



countryname
People's Republic of China         19
Republic of Indonesia              19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
Nepal                              12
People's Republic of Bangladesh    12
Kingdom of Morocco                 12
Africa                             11
Republic of Mozambique             11
Name: project_name, dtype: int64

In [5]:
#get top 10 project themes using 'mjtheme_namecode' column
#print(wb[['mjtheme_namecode']].head())
wb2 = json.load(open('data/world_bank_projects.json'))
wb2 = json_normalize(wb2, 'mjtheme_namecode')
wb10pt = wb2.groupby('name')['code'].count().sort_values(ascending=False).head(10)

print(wb10pt)

#Alternative code
#y = wb2.name.value_counts()
#print(y)

name
Environment and natural resources management    223
Rural development                               202
Human development                               197
Public sector governance                        184
Social protection and risk management           158
Financial and private sector development        130
                                                122
Social dev/gender/inclusion                     119
Trade and integration                            72
Urban development                                47
Name: code, dtype: int64


In [12]:
#3. In 2. above you will notice that some entries have only the code and the name is missing. 
# Create a dataframe with the missing names filled in.
wb3 = json.load(open('data/world_bank_projects.json'))
wb3 = json_normalize(wb3, 'mjtheme_namecode', 'id')
wb3 = wb3[['id','code','name']]
wb_ntc = wb3.groupby('name').code.max().sort_values(ascending=False)



In [3]:
# create lookup table from code to name
#wb_ntc = wb3.groupby('name').code.max().sort_values(ascending=False)
#print(wb_ntc.head())


# drop the missing name rows
wb_ntc = wb_ntc[wb_ntc.index != '']

#print(wb_ntc.head())
# convert to dataframe
df_wb_ntc = pd.DataFrame(wb_ntc,columns=['code'])
#print(df_wb_ntc.head())

# pull name into a column
df_wb_ntc['name_clean'] = df_wb_ntc.index

# set code to be the index
df_wb_ntc = df_wb_ntc.set_index(['code'])

# now merge on the name based on the code for the missing projects
#print(wb3.shape)
wb_final = wb3.merge(df_wb_ntc,how='outer',left_on=['code'],right_index=True)
#print(wb_final.shape)
#print(wb_final.head(10))
print('')
wb_final['name'] = wb_final['name_clean']
del wb_final['name_clean']
#print('')
print(wb_final)
#there is a one to many relationship between name and code and id (one id has multiple themes)
#print(wb3.head())
#print('')
#print(wb3.groupby('id').code.nunique().sort_values(ascending=False).head(10))

,id,code,name
0,P129828,8,Human development
1,P129828,11,
2,P144674,1,Economic management
3,P144674,6,Social protection and risk management
4,P145310,5,Trade and integration
5,P145310,2,Public sector governance
6,P145310,11,Environment and natural resources management
7,P145310,6,Social protection and risk management
8,P144665,7,Social dev/gender/inclusion
9,P144665,7,Social dev/gender/inclusion


In [7]:
#simpler way: use dictionaries
#key is code column, value is project name. loop through name. if name exists, continue
#else assign dict value (project name) to missing entry
wb3 = json.load(open('data/world_bank_projects.json'))
wb3 = json_normalize(wb3, 'mjtheme_namecode', 'id')

mapp = wb3[['code','name']]

mapp = mapp[mapp['name']!='']

mapp = mapp.sort_values('code')
mapp = mapp.drop_duplicates()

map_dict = mapp.set_index('code').to_dict()
#map_dict = map.to_dict()

map_dict = map_dict['name']


#print(map_dict)

#print(map_dict['name']) #this is type dictionary

#for code, name in map_dict.items(): #this code worked!
#    print(code, name)

#for i, row in wb3.iterrows():
#    if row[‘name’]==‘’:
#        row[‘name’] = mapp[row[‘code’]]


#wb3['name'] = wb3.apply(lambda row: row['name'] if row['name']!='' else map_dict[row['code']]  , axis=1)
#print(wb3.head())

0
code                    8
name    Human development
id                P129828
Name: 0, dtype: object
1
code         11
name           
id      P129828
Name: 1, dtype: object
2
code                      1
name    Economic management
id                  P144674
Name: 2, dtype: object
3
code                                        6
name    Social protection and risk management
id                                    P144674
Name: 3, dtype: object
4
code                        5
name    Trade and integration
id                    P145310
Name: 4, dtype: object
5
code                           2
name    Public sector governance
id                       P145310
Name: 5, dtype: object
6
code                                              11
name    Environment and natural resources management
id                                           P145310
Name: 6, dtype: object
7
code                                        6
name    Social protection and risk management
id                                 